
# Querying with Dataframes

Apache Spark&trade; allow you to use DataFrames to query large data files.

## In this lesson you:
* Learn about Spark DataFrames.
* Query large files using Spark DataFrames.


## Audience
* Primary Audience: Data Engineers and Data Scientists
* Secondary Audience: Data Analysts

## Prerequisites
* Web browser: Chrome or Firefox



### Introducing DataFrames

Under the covers, DataFrames are derived from data structures known as Resilient Distributed Datasets (RDDs). RDDs and DataFrames are immutable distributed collections of data. Let's take a closer look at what some of these terms mean before we understand how they relate to DataFrames:

* **Resilient**: They are fault tolerant, so if part of your operation fails, Spark  quickly recovers the lost computation.
* **Distributed**: RDDs are distributed across networked machines known as a cluster.
* **DataFrame**: A data structure where data is organized into named columns, like a table in a relational database, but with richer optimizations under the hood. 

Without the named columns and declared types provided by a schema, Spark wouldn't know how to optimize the executation of any computation. Since DataFrames have a schema, they use the Catalyst Optimizer to determine the optimal way to execute your code.

DataFrames were invented because the business community uses tables in a relational database, Pandas or R DataFrames, or Excel worksheets. A Spark DataFrame is conceptually equivalent to these, with richer optimizations under the hood and the benefit of being distributed across a cluster.

#### Interacting with DataFrames

Once created (instantiated), a DataFrame object has methods attached to it. Methods are operations one can perform on DataFrames such as filtering,
counting, aggregating and many others.

> <b>Example</b>: To create (instantiate) a DataFrame, use this syntax: `df = ...`

To display the contents of the DataFrame, apply a `show` operation (method) on it using the syntax `df.show()`. 

The `.` indicates you are *applying a method on the object*.

In working with DataFrames, it is common to chain operations together, such as: `df.select().filter().orderBy()`.  

By chaining operations together, you don't need to save intermediate DataFrames into local variables (thereby avoiding the creation of extra objects).

Also note that you do not have to worry about how to order operations because the optimizier determines the optimal order of execution of the operations for you.

`df.select(...).orderBy(...).filter(...)`

versus

`df.filter(...).select(...).orderBy(...)`


#### DataFrames and SQL

DataFrame syntax is more flexible than SQL syntax. Here we illustrate general usage patterns of SQL and DataFrames.

Suppose we have a data set we loaded as a table called `myTable` and an equivalent DataFrame, called `df`.
We have three fields/columns called `col_1` (numeric type), `col_2` (string type) and `col_3` (timestamp type)
Here are basic SQL operations and their DataFrame equivalents. 

Notice that columns in DataFrames are referenced by `col("<columnName>")`.

| SQL                                         | DataFrame (Python)                    |
| ------------------------------------------- | ------------------------------------- | 
| `SELECT col_1 FROM myTable`                 | `df.select(col("col_1"))`             | 
| `DESCRIBE myTable`                          | `df.printSchema()`                    | 
| `SELECT * FROM myTable WHERE col_1 > 0`     | `df.filter(col("col_1") > 0)`         | 
| `..GROUP BY col_2`                          | `..groupBy(col("col_2"))`             | 
| `..ORDER BY col_2`                          | `..orderBy(col("col_2"))`             | 
| `..WHERE year(col_3) > 1990`                | `..filter(year(col("col_3")) > 1990)` | 
| `SELECT * FROM myTable LIMIT 10`            | `df.limit(10)`                        |
| `display(myTable)` (text format)            | `df.show()`                           | 
| `display(myTable)` (html format)            | `display(df)`                         |

**Hint:** You can also run SQL queries with the special syntax `spark.sql("SELECT * FROM myTable")`

In this course you see many other usages of DataFrames. It is left up to you to figure out the SQL equivalents 
(left as exercises in some cases).

### Querying Data 
This lesson uses the `2017_2018_NBA_Salaries.csv` data set, which is in CSV format.


In [13]:
import pyspark

nbaDF = spark.read.csv("C:\Users\<userName>\Desktop\demoNotes\2017_2018_NBA_Salaries.csv")


# CSV files read 
nbaDF = (spark
  .read                                                # Call the read method returning a DataFrame
  .option("inferSchema","true")                        # Option to tell Spark to infer the schema
  .option("header","true")                             # Option telling Spark that the file has a header
  .csv("C:\Users\<userName>\Desktop\demoNotes\2017_2018_NBA_Salaries.csv"))  # Option telling Spark where the file is

Note: the method to read .CSV file spark.read.csv
Spark provides special methods to read files in different format such as AVRO, PARQUET etc.
spark.read.avro
spark.read.parquet

Take a look at the schema with the `printSchema` method. This tells you the field name, field type, and whether the column is nullable or not (default is true).

In [15]:
nbaDF.printSchema()

Now the DataFrame is created, view its contents by invoking the show method.

By default, show() (without any parameters) prints the first 20 rows.

Print the top n rows by invoking show(n)


In [ ]:
nbaDF.show(10)

Answer the following question:
> According to our data, list out the players in USA

Use the DataFrame `select` and `filter` methods.

In [17]:
from pyspark.sql.functions import *

nbaDF.select("Palyer", "NBA_Country", "Salary", "Age").filter("NBA_Country = 'USA'")

### Built-In Functions

Spark provides a number of <a href="https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.functions$" target="_blank">built-in functions</a>, many of which can be used directly with DataFrames.  Use these functions in the `filter` expressions to filter data and in `select` expressions to create derived columns.



How many Players are there in USA?

In [24]:
from pyspark.sql.functions import *

nbaDF.filter("NBA_Country = 'USA'").count()

Who is the youngest player USA?

In [ ]:
from pyspark.sql.functions import *

nbaDF.select("Palyer", "NBA_Country", "Salary", "Age").filter("NBA_Country = 'USA'").orderBy("Age").limit(1)

### Temporary Views

In DataFrames, <b>temporary views</b> are used to make the DataFrame available to SQL, and work with SQL syntax seamlessly.

A temporary view gives you a name to query from SQL, but unlike a table it exists only for the duration of your Spark Session. As a result, the temporary view will not carry over when you restart the cluster or switch to a new notebook. It also won't show up in the Data button on the menu on the left side of a Databricks notebook which provides easy access to databases and tables.

The statement in the following cells create a temporary view containing the same data.

In [32]:
from pyspark.sql.functions import *

nbaDF.createOrReplaceTempView("nbaPlayers")

To view the contents of temporary view, use select notation.

In [34]:
from pyspark.sql.functions import *

spark.sql("SELECT * FROM  nbaPlayers WHERE NBA_Country = 'USA' ")

In [38]:
from pyspark.sql.functions import *

spark.sql("SELECT COUNT(1) FROM  nbaPlayers WHERE NBA_Country = 'USA' ")

In [42]:
from pyspark.sql.functions import *

spark.sql("SELECT NBA_Country, COUNT(1) FROM nbaPlayers GROUP BY NBA_Country")

In [ ]:
from pyspark.sql.functions import *

nbaDF.groupBy("NBA_Country").count()

## Column Alias

In [ ]:
from pyspark.sql.functions import *

nbaDF.select("Palyer", "NBA_Country", "Salary", col("Age").alias("Player_Age")).filter("NBA_Country = 'USA'")

## Basic Aggregations

Using built-in Spark functions</a>, you can aggregate data in various ways. 

Run the cell below to compute the average of all salaries in the DataFrame for USA.

By default, you get a floating point value.

In [ ]:
from pyspark.sql.functions import *

nbaDF.select(avg("Salary").alias("averageSalary")).filter("NBA_Country = 'USA'")

In [ ]:
from pyspark.sql.functions import *

nbaDF..select(max("Salary").alias("max"), min("Salary").alias("min"), round(avg("Salary")).alias("averageSalary")).filter("NBA_Country = 'USA'")

## Review Questions

**Q:** What Scala keyword is used to *create* a DataFrame object?  
**A:** An object is created by invoking the `val` keyword, so `val myDataFrameDF`. 

**Q:** What methods (operations) can you perform on a DataFrame object?  
**A:** The full list is here: <a href="http://spark.apache.org/docs/2.0.0/api/python/pyspark.sql.html" target="_blank"> pyspark.sql module</a>

**Q:** Why do you chain methods (operations) `myDataFrameDF.select().filter().groupBy()`?  
**A:** To avoid the creation of temporary DataFrames as local variables. 

For example, you could have written the above as 
`val tempDF1 = myDataFrameDF.select()`,  `val tempDF2 = tempDF1.filter()` and
then `tempDF2.groupBy()`. 

This is syntatically equivalent, but, notice how you now have extra local variables.

**Q:** What is the DataFrame syntax to create a temporary view?    
**A:** ```myDF.createOrReplaceTempView("MyTempView")```

**Q:** What is the DataFrame equivalent of the SQL statement SELECT count(*) AS total
**A:**.agg(count("*").alias("total"))

## Additional Topics & Resources

* <a href="https://spark.apache.org/docs/latest/sql-programming-guide.html" target="_blank">Spark SQL, DataFrames and Datasets Guide</a>